In [1]:
import pandas as pd
import pickle

import nltk
from nltk.stem import PorterStemmer

In [2]:
from openai import OpenAI
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path
import sys# make data_path pathlib-friendly and resolve relative to this file
code_path = Path("../code")
# if not code_path.is_absolute():
#     # resolve relative to the repository/code file location (works on Windows)
#     code_path = (Path(__file__).resolve().parent / code_path).resolve()

sys.path.append(str(code_path))
from text_preprocessing import preprocess_sentence


MODEL = os.getenv("LMSTUDIO_MODEL")
client = OpenAI(
    base_url=os.getenv("LMSTUDIO_BASE_URL"),
    api_key="lm-studio"
)
def get_embedding(text, model="model-identifier"):
   text = text.replace("\n", " ")   
   embedding = client.embeddings.create(input=[text], model=MODEL).data[0].embedding
   return np.array(embedding)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\achil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\achil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
data = pd.read_csv('../data/data.csv')

In [4]:
dataframe = {'code' : [], 'symptome' : []}

In [5]:
for i in data.index : 
    text = data.description[i].split('..')
    for t in text : 
        t = preprocess_sentence(t)
        dataframe['code'].append(data.code[i])
        dataframe['symptome'].append(t)

database = pd.DataFrame(dataframe)
database["code"] = database["code"].str.split('_').str[0]

# prepare disorder names
raw_data_file = "../data/33clusters30PCs_withNames.csv"
raw_data = pd.read_csv(raw_data_file, index_col=None)
 # Split pipe-separated values in 'code' into separate rows, repeating other columns,
raw_data['code'] = raw_data['code'].astype(str)
raw_data['code_list'] = raw_data['code'].str.split('|')
expanded = raw_data.explode('code_list').copy()
expanded['code'] = expanded['code_list'].str.strip()
expanded = expanded.drop(columns=['code_list']).reset_index(drop=True)
# now `expanded` has one row per code; assign back if you want to replace raw_data,
raw_data = expanded

raw_data['code_key'] = raw_data['code'].astype(str).str.extract(r'(^[A-Z]\d{2}(?:\.\d+)?)', expand=False)

# build mapping from code_key -> diagnosis (take first if duplicates)
mapping = raw_data.groupby('code_key')['diagnosis'].first().to_dict()

# map to database (database['code'] should already be cleaned like "F70" or "F60.2")
database['disorder'] = database['code'].map(mapping)
database = database[['code', 'disorder', 'symptome']]
# optional: see how many didn't match
print(f"unmatched codes: {database['disorder'].isna().sum()}")



print(f"number of symptoms before deduplication: {len(database)}")
# drop duplicates to avoid redundant symptoms
database = database.drop_duplicates(subset=["symptome"]).reset_index(drop=True)
print(f"number of symptoms after deduplication: {len(database)}")
database.to_csv('../data/datalong.csv')

unmatched codes: 98
number of symptoms before deduplication: 1554
number of symptoms after deduplication: 807


In [6]:
list_symptoms = pd.read_csv('../data/datalong.csv', index_col=0)
symptoms_embeds = {"disorder" : [], "symptome" : [], "embedding" : []}

In [7]:
for i in list_symptoms.index : 
    symptoms_embeds['disorder'].append(list_symptoms.disorder[i])
    symptoms_embeds['symptome'].append(list_symptoms.symptome[i])
    symptoms_embeds['embedding'].append(get_embedding(list_symptoms.symptome[i]))

In [8]:
symptoms_embeddings = pd.DataFrame(symptoms_embeds)

In [9]:
with open('../data/symptomes_embedding.pkl', 'wb') as fp:
        pickle.dump(symptoms_embeddings, fp)

In [10]:
# repeat using BioSentVec model
import sent2vec
model_path = "../models/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')


AttributeError: module 'sent2vec' has no attribute 'Sent2vecModel'